# 제품 이상여부 판별 프로젝트


### 데이터 읽어오기


In [1]:
import os
from pprint import pprint

import optuna
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

RANDOM_STATE = 110

train_data = pd.read_csv("train_data_0827.csv")
test_data = pd.read_csv("test_data_0827.csv")

In [3]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Receip_No_encoded',
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result',
    'WorkMode Collect Result'
]

In [4]:
# 전체 공통 변수
### train
var_all_train = [
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

In [5]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [6]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [7]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [8]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [9]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 40
train_data_dam DataFrame의 칼럼 수: 23
train_data_autoclave DataFrame의 칼럼 수: 8
train_data_fill1 DataFrame의 칼럼 수: 14
train_data_fill2 DataFrame의 칼럼 수: 14
----test data-----
test_data DataFrame의 칼럼 수: 41
test_data_dam DataFrame의 칼럼 수: 24
test_data_autoclave DataFrame의 칼럼 수: 9
test_data_fill1 DataFrame의 칼럼 수: 15
test_data_fill2 DataFrame의 칼럼 수: 15


---

## Optuna

스레스홀드 0.3으로 맞춘상태에서 튜닝 진행한 것

In [12]:
RANDOM_STATE = 110
THRESHOLD = 0.3

In [12]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

def objectiveCatBoost(trial, x_tr, y_tr, x_val, y_val):
    
    # 'Normal'과 'AbNormal'을 숫자로 변환
    y_tr = y_tr.map({'Normal': 0, 'AbNormal': 1})
    y_val = y_val.map({'Normal': 0, 'AbNormal': 1})
    
    param = {
        'iterations': trial.suggest_int('iterations', 800, 5000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.2),
        'depth': trial.suggest_int('depth', 4, 13),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 5),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10, log=True),
        'random_strength': trial.suggest_float('random_strength', 0, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.1, 10),
        'border_count': trial.suggest_int('border_count', 128, 300),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        
        'random_seed': RANDOM_STATE,
        'eval_metric': 'F1',
        'logging_level': 'Silent',
        'boosting_type': 'Plain'
    }

    model = CatBoostClassifier(**param)
    model.fit(x_tr, y_tr)
    pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data_dam.drop("target", axis=1),  
    train_data_dam["target"],               
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study.optimize(lambda trial: objectiveCatBoost(trial, x_train, y_train, x_val, y_val), n_trials=300)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2024-08-27 12:50:37,154] A new study created in memory with name: no-name-8e12b553-d3fe-42d7-96ee-dd37952d732a
[I 2024-08-27 12:50:50,218] Trial 0 finished with value: 0.20231822971549 and parameters: {'iterations': 1287, 'learning_rate': 0.13203262096748453, 'depth': 7, 'min_data_in_leaf': 4, 'l2_leaf_reg': 4.780689663026719, 'random_strength': 6.824631292225899, 'bagging_temperature': 9.554598960277284, 'border_count': 238, 'scale_pos_weight': 5.61649246971856, 'grow_policy': 'Depthwise'}. Best is trial 0 with value: 0.20231822971549.
[I 2024-08-27 12:50:58,585] Trial 1 finished with value: 0.2258630778232885 and parameters: {'iterations': 1368, 'learning_rate': 0.0030110113610709835, 'depth': 8, 'min_data_in_leaf': 3, 'l2_leaf_reg': 1.0505036749294598, 'random_strength': 5.595100695350062, 'bagging_temperature': 0.8684052021612866, 'border_count': 198, 'scale_pos_weight': 3.8269447050797387, 'grow_policy': 'SymmetricTree'}. Best is trial 1 with value: 0.2258630778232885.
[I 2024-

[I 2024-08-27 13:04:45,371] Trial 19 finished with value: 0.21220159151193632 and parameters: {'iterations': 891, 'learning_rate': 0.1049877907247948, 'depth': 11, 'min_data_in_leaf': 3, 'l2_leaf_reg': 2.2330781016358237, 'random_strength': 5.979128380609728, 'bagging_temperature': 2.591693980646216, 'border_count': 204, 'scale_pos_weight': 3.6195532580215306, 'grow_policy': 'Depthwise'}. Best is trial 1 with value: 0.2258630778232885.
[I 2024-08-27 13:04:58,149] Trial 20 finished with value: 0.2050663449939686 and parameters: {'iterations': 1499, 'learning_rate': 0.06375659123671426, 'depth': 7, 'min_data_in_leaf': 2, 'l2_leaf_reg': 3.3084734065478965, 'random_strength': 0.07499004497755202, 'bagging_temperature': 6.0701375779643065, 'border_count': 239, 'scale_pos_weight': 2.1163557278053498, 'grow_policy': 'Lossguide'}. Best is trial 1 with value: 0.2258630778232885.
[I 2024-08-27 13:05:14,812] Trial 21 finished with value: 0.20466321243523314 and parameters: {'iterations': 818, 'le

[I 2024-08-27 13:10:16,013] Trial 38 finished with value: 0.2268326417704011 and parameters: {'iterations': 2149, 'learning_rate': 0.015841884870864215, 'depth': 6, 'min_data_in_leaf': 5, 'l2_leaf_reg': 2.8262056920496033, 'random_strength': 4.828585959274561, 'bagging_temperature': 8.601233154627486, 'border_count': 212, 'scale_pos_weight': 1.8388105800654793, 'grow_policy': 'Lossguide'}. Best is trial 29 with value: 0.2285714285714286.
[I 2024-08-27 13:10:26,372] Trial 39 finished with value: 0.2009273570324575 and parameters: {'iterations': 1405, 'learning_rate': 0.011400236830095955, 'depth': 4, 'min_data_in_leaf': 5, 'l2_leaf_reg': 2.866956300989676, 'random_strength': 5.994140607020709, 'bagging_temperature': 8.636097739199696, 'border_count': 256, 'scale_pos_weight': 1.924878596245392, 'grow_policy': 'Lossguide'}. Best is trial 29 with value: 0.2285714285714286.
[I 2024-08-27 13:10:42,692] Trial 40 finished with value: 0.21731748726655348 and parameters: {'iterations': 1699, 'le

[I 2024-08-27 13:14:30,011] Trial 57 finished with value: 0.21717171717171715 and parameters: {'iterations': 922, 'learning_rate': 0.02332076398268565, 'depth': 8, 'min_data_in_leaf': 2, 'l2_leaf_reg': 1.7693092795349226, 'random_strength': 1.5468808627885056, 'bagging_temperature': 7.762024545080468, 'border_count': 215, 'scale_pos_weight': 2.298446851450045, 'grow_policy': 'Lossguide'}. Best is trial 29 with value: 0.2285714285714286.
[I 2024-08-27 13:14:34,486] Trial 58 finished with value: 0.20712510356255182 and parameters: {'iterations': 817, 'learning_rate': 0.03347450119250588, 'depth': 7, 'min_data_in_leaf': 3, 'l2_leaf_reg': 2.1099037236155, 'random_strength': 2.2929684597587294, 'bagging_temperature': 6.517373699624093, 'border_count': 235, 'scale_pos_weight': 3.294911695697384, 'grow_policy': 'SymmetricTree'}. Best is trial 29 with value: 0.2285714285714286.
[I 2024-08-27 13:15:16,982] Trial 59 finished with value: 0.20448179271708686 and parameters: {'iterations': 4957, 'l

[I 2024-08-27 13:19:39,030] Trial 76 finished with value: 0.21024967148488832 and parameters: {'iterations': 1331, 'learning_rate': 0.02906898000318176, 'depth': 8, 'min_data_in_leaf': 3, 'l2_leaf_reg': 2.913138284312535, 'random_strength': 2.5941243793085795, 'bagging_temperature': 1.8499032124827028, 'border_count': 149, 'scale_pos_weight': 2.0040795081652334, 'grow_policy': 'Lossguide'}. Best is trial 29 with value: 0.2285714285714286.
[I 2024-08-27 13:19:50,834] Trial 77 finished with value: 0.21754042136207746 and parameters: {'iterations': 1169, 'learning_rate': 0.0013434301115163687, 'depth': 6, 'min_data_in_leaf': 4, 'l2_leaf_reg': 1.2300309056711305, 'random_strength': 4.3025992622393705, 'bagging_temperature': 7.6199476653024, 'border_count': 205, 'scale_pos_weight': 3.8489026085375, 'grow_policy': 'Lossguide'}. Best is trial 29 with value: 0.2285714285714286.
[I 2024-08-27 13:20:05,369] Trial 78 finished with value: 0.21715526601520085 and parameters: {'iterations': 1571, 'l

...

[I 2024-08-27 14:30:30,952] Trial 228 finished with value: 0.22972972972972971 and parameters: {'iterations': 1726, 'learning_rate': 0.0043995707572901405, 'depth': 11, 'min_data_in_leaf': 2, 'l2_leaf_reg': 1.0044709403284837, 'random_strength': 0.5109582506178085, 'bagging_temperature': 4.1104504613626665, 'border_count': 164, 'scale_pos_weight': 1.1451111398424243, 'grow_policy': 'Depthwise'}. Best is trial 206 with value: 0.24382716049382716.
[I 2024-08-27 14:31:08,424] Trial 229 finished with value: 0.2239031770045386 and parameters: {'iterations': 1598, 'learning_rate': 0.016028000750929205, 'depth': 11, 'min_data_in_leaf': 2, 'l2_leaf_reg': 1.097882195927194, 'random_strength': 0.3074894460834856, 'bagging_temperature': 3.3516409581025015, 'border_count': 145, 'scale_pos_weight': 1.0137974637950349, 'grow_policy': 'Depthwise'}. Best is trial 206 with value: 0.24382716049382716.
[I 2024-08-27 14:31:38,609] Trial 230 finished with value: 0.20926243567753 and parameters: {'iteration

[I 2024-08-27 14:41:06,142] Trial 247 finished with value: 0.22188449848024316 and parameters: {'iterations': 1517, 'learning_rate': 0.011270709350121444, 'depth': 11, 'min_data_in_leaf': 2, 'l2_leaf_reg': 1.115773595949947, 'random_strength': 0.003768542051994339, 'bagging_temperature': 4.0273878819062485, 'border_count': 144, 'scale_pos_weight': 1.2252876059110869, 'grow_policy': 'Depthwise'}. Best is trial 232 with value: 0.24962406015037594.
[I 2024-08-27 14:42:39,575] Trial 248 finished with value: 0.23008849557522124 and parameters: {'iterations': 4130, 'learning_rate': 0.01807721915685444, 'depth': 12, 'min_data_in_leaf': 2, 'l2_leaf_reg': 1.0418852149140418, 'random_strength': 0.27401883481858613, 'bagging_temperature': 4.263713786869621, 'border_count': 136, 'scale_pos_weight': 1.006866250382095, 'grow_policy': 'Depthwise'}. Best is trial 232 with value: 0.24962406015037594.
[I 2024-08-27 14:43:28,141] Trial 249 finished with value: 0.23462088698140202 and parameters: {'iterat

[I 2024-08-27 14:54:54,905] Trial 266 finished with value: 0.22038567493112948 and parameters: {'iterations': 2705, 'learning_rate': 0.01042059218077111, 'depth': 11, 'min_data_in_leaf': 2, 'l2_leaf_reg': 1.1293713897436926, 'random_strength': 0.9527474277894021, 'bagging_temperature': 3.3639557221330842, 'border_count': 132, 'scale_pos_weight': 1.6952504506367143, 'grow_policy': 'Depthwise'}. Best is trial 232 with value: 0.24962406015037594.
[I 2024-08-27 14:55:41,326] Trial 267 finished with value: 0.24216524216524213 and parameters: {'iterations': 1301, 'learning_rate': 0.00712640834617437, 'depth': 13, 'min_data_in_leaf': 2, 'l2_leaf_reg': 1.0346422052697093, 'random_strength': 0.39981926000637474, 'bagging_temperature': 3.637593094562944, 'border_count': 143, 'scale_pos_weight': 1.6527165320941561, 'grow_policy': 'Depthwise'}. Best is trial 232 with value: 0.24962406015037594.
[I 2024-08-27 14:56:27,140] Trial 268 finished with value: 0.24122137404580152 and parameters: {'iterati

[I 2024-08-27 15:09:59,119] Trial 285 finished with value: 0.23580786026200873 and parameters: {'iterations': 1408, 'learning_rate': 0.0067910594894665405, 'depth': 13, 'min_data_in_leaf': 2, 'l2_leaf_reg': 1.111962667660076, 'random_strength': 0.3426771812639971, 'bagging_temperature': 3.2111823423808454, 'border_count': 291, 'scale_pos_weight': 1.6474660988243612, 'grow_policy': 'Depthwise'}. Best is trial 232 with value: 0.24962406015037594.
[I 2024-08-27 15:10:42,091] Trial 286 finished with value: 0.23262839879154076 and parameters: {'iterations': 1268, 'learning_rate': 0.014020517498485312, 'depth': 13, 'min_data_in_leaf': 2, 'l2_leaf_reg': 1.2160189121901877, 'random_strength': 1.1540071694912244, 'bagging_temperature': 2.548518222789731, 'border_count': 239, 'scale_pos_weight': 1.331837727737883, 'grow_policy': 'Depthwise'}. Best is trial 232 with value: 0.24962406015037594.
[I 2024-08-27 15:11:12,954] Trial 287 finished with value: 0.20774647887323944 and parameters: {'iterati

Best trial: score 0.24962406015037594, 
params {'iterations': 1478, 'learning_rate': 0.009068953796649421, 'depth': 11, 'min_data_in_leaf': 2, 'l2_leaf_reg': 1.187291687951122, 'random_strength': 0.43102541391012816, 'bagging_temperature': 3.1790702578164853, 'border_count': 155, 'scale_pos_weight': 1.4418307437388553, 'grow_policy': 'Depthwise'}


1. 0.24962406015037594

params {'iterations': 1478, 'learning_rate': 0.009068953796649421, 'depth': 11, 'min_data_in_leaf': 2, 'l2_leaf_reg': 1.187291687951122, 'random_strength': 0.43102541391012816, 'bagging_temperature': 3.1790702578164853, 'border_count': 155, 'scale_pos_weight': 1.4418307437388553, 'grow_policy': 'Depthwise'}

In [13]:
# 모델 설정 및 하이퍼파라미터
models = {
    'et': ExtraTreesClassifier(),
    'rf': RandomForestClassifier(),
    'cat': CatBoostClassifier(),
    'lgbm': LGBMClassifier(),
    'xgb': XGBClassifier(),
    'dt': DecisionTreeClassifier()
}

def train_and_evaluate_model(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return
    
    # 데이터셋 분할
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # 모델 선택
    model = models[model_name]

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(x_train, y_train)

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    # 예측
    y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'{model_name} 모델이 {data_name} 데이터로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

In [14]:
train_and_evaluate_model(
    'cat', train_data_dam,
    iterations = 1478, 
    learning_rate = 0.009068953796649421, 
    depth = 11, 
    min_data_in_leaf = 2,
    l2_leaf_reg = 1.187291687951122,
    random_strength = 0.43102541391012816, 
    bagging_temperature = 3.1790702578164853, 
    border_count = 155, 
    scale_pos_weight = 1.4418307437388553,
    grow_policy = 'Depthwise',

    random_state = RANDOM_STATE,
    eval_metric = 'F1',
    logging_level = 'Silent',
    boosting_type = 'Plain'
)

cat 모델이 train_data_dam 데이터로 학습한 결과:
F1 Score: 0.25000000000000006
---
Confusion Matrix:
[[7514  148]
 [ 356   84]]
---
Accuracy: 0.9377931374969144
Precision: 0.3620689655172414
Recall: 0.19090909090909092




In [ ]:
'''
cat 모델이 train_data_dam 데이터로 학습한 결과:
F1 Score: 0.24962406015037594
---
Confusion Matrix:
[[7520  142]
 [ 357   83]]
---
Accuracy: 0.9384102690693656
Precision: 0.3688888888888889
Recall: 0.18863636363636363
'''